In [3]:
import torch
torch.__version__

'1.7.1+cu110'

In [1]:
import csv

# 打开CSV文件
with open('/mnt/mydisk/beijing.csv', 'r') as f:
    # 创建CSV文件读取器
    reader = csv.reader(f)
    # 获取第一行数据
    header = next(reader)
    # 将label加入到第一行第三列
    header.insert(0, 'label')
    # 打开一个新文件
    with open('new_file.csv', 'w', newline='') as fw:
        # 创建CSV文件写入器
        writer = csv.writer(fw)
        # 写入第一行数据
        writer.writerow(header)
        # 遍历每一行数据
        for i, row in enumerate(reader):
            # 在每一行数据前添加一个编号
            new_row = [i+1] + row
            # 写入新的一行数据
            writer.writerow(new_row)


In [ ]:
#快速演示
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device=', device)

from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('/mnt/mydisk/bert-base-chinese')
#需要移动到cuda上
pretrained.to(device)

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)


#定义下游任务模型
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out


model = Model()
#同样要移动到cuda
model.to(device)

#虚拟一批数据,需要把所有的数据都移动到cuda上
input_ids = torch.ones(16, 100).long().to(device)
attention_mask = torch.ones(16, 100).long().to(device)
token_type_ids = torch.ones(16, 100).long().to(device)
labels = torch.ones(16).long().to(device)

#试算
model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

#后面的计算和中文分类完全一样，只是放在了cuda上计算

# Start


In [19]:
import pandas as pd
import torch.utils.data as data

class CSVDataset(data.Dataset):
    def __init__(self, csv_file_path, text_column, label_column):
        self.data = pd.read_csv(csv_file_path)
        self.text_column = text_column
        self.label_column = label_column

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx][self.text_column]
        label = self.data.iloc[idx][self.label_column]
        return text, label
    

dataset = CSVDataset('/mnt/mydisk/CSTD/new_file.csv', 'title', 'label')
len(dataset)
for i in dataset:
    print(i)

('共青团北京市第十五次代表大会开幕 尹力讲话 贺军科殷勇魏小东刘伟出席', 1)
('尹力到市轨道交通指挥中心调研时强调 构建立体化现代化城市交通系统 更好满足城市发展和群众出行需求', 2)
('“五一”假期首日，尹力殷勇走进火车站、商圈检查安全生产和城市运行保障等工作，并慰问坚守岗位的劳动者！', 3)
('“五一”假期北京预计接待游客885万人次 各大热门公园景区基本约满', 4)
('市委常委会召开会议 听取全市社会建设工作情况汇报 研究“五一”安全防范和假日相关工作等事项 市委书记尹力主持会议', 5)
('市推进全国文化中心建设领导小组会议召开 尹力主持 殷勇出席', 6)
('尹力调研中轴线申遗保护工作时强调 让北京这座千年古都焕发出更加蓬勃的时代活力', 7)
('本市召开全市领导干部会议 尹力强调 深刻反思汲取教训 时刻保持警钟长鸣 全面开展全市安全生产隐患大排查大整治 殷勇刘伟出席', 8)
('平谷桃花节办了30余年 从带货大桃到带火文旅——桃谷花事', 9)
('市“两区”工作领导小组召开会议 尹力主持 殷勇刘伟出席', 10)
('尹力到市发展改革委、市财政局、市税务局、市统计局调研时强调 把主题教育学习成果转化为推动首都高质量发展实效', 11)
('市委常委会召开会议 传达学习习近平总书记重要讲话精神 研究部署全市深入开展主题教育工作等事项 市委书记尹力主持会议', 12)
('春季游热度攀升 旅游市场加速复苏', 13)
('市委城市工作委员会召开全体会议 尹力主持 殷勇刘伟出席', 14)
('书香浸古建 激活城市文化记忆', 15)
('博物馆之城 让金名片更加闪耀', 16)
('市委常委会召开会议 研究加快推动北京国际科技创新中心建设工作方案等事项 市委书记尹力主持会议', 17)
('“文化润疆”打造文化名片 《五星出东方》将在和田驻场演出', 18)
('尹力在金融工作座谈会上强调 贯彻落实党的二十大精神 推动首都金融高质量发展 易纲郭树清易会满殷勇参加', 19)
('三条文化带 承载古都“城市之魂”', 20)
('“中国发展高层论坛2023年年会”北京市招待晚宴举行 尹力致辞 陆昊殷勇刘伟出席', 21)
('守护好北京历史文脉的根与魂', 22)
('城市副中心大运河游船今春首航 多条航线饱览千年运河风光', 2

In [ ]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')


def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = torch.LongTensor(labels).to(device)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

In [ ]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 5 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)

    if i == 100:
        break

In [ ]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 5:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    print(correct / total)


test()